In [23]:
import os
import tensorflow as tf
os.environ['KERAS_BACKEND'] = 'tensorflow'

import sys
import random
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
import skimage

import tensorflow as tf

from keras.preprocessing.image import array_to_img, img_to_array, load_img
        
PATH = 'input'

train_df = pd.read_csv('input/train.csv')
train_ids = train_df.id.values

sample_df = pd.read_csv('input/sample_submission.csv')
test_ids = sample_df.id.values

In [48]:
im_width = 128
im_height = 128
im_chan = 4

def load_and_resize(path):
    img = load_img(path)  
    x = img_to_array(img)[:,:,1]
    if x.shape != (101, 101):
        print(path, shape)
    x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)
    return x

# Get and resize train images and masks
X_train = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.float32)
Y_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')

for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):    
    x = load_and_resize(os.path.join(PATH, 'train', 'images', "%s.png" % id_))
    x = x[..., 0]
    
    Y_train[n] = load_and_resize(os.path.join(PATH, 'train', 'masks', "%s.png" % id_))
        
    X_train[n, ..., 0] = x
    X_train[n, ..., 1] = x.cumsum(axis=0)
    X_train[n, ..., 2] = skimage.filters.sobel_v(x)
    X_train[n, ..., 3] = skimage.filters.sobel_h(x)
        
print('Done!')

  0%|          | 0/4000 [00:00<?, ?it/s]/opt/miniconda2/envs/avito/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
  0%|          | 17/4000 [00:00<00:24, 162.31it/s]

Getting and resizing train images and masks ... 


100%|██████████| 4000/4000 [00:14<00:00, 267.92it/s]

Done!


In [50]:
# Get and resize test images
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.float32)

print('Getting and resizing test images ... ')

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    x = load_and_resize(os.path.join(PATH, 'test', 'images', '%s.png' % id_))
    x = x[..., 0]
    
    X_test[n, ..., 0] = x
    X_test[n, ..., 1] = x.cumsum(axis=0)
    X_test[n, ..., 2] = skimage.filters.sobel_v(x)
    X_test[n, ..., 3] = skimage.filters.sobel_h(x)


  0%|          | 0/18000 [00:00<?, ?it/s]/opt/miniconda2/envs/avito/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
  0%|          | 36/18000 [00:00<00:50, 356.40it/s]

Getting and resizing test images ... 


100%|██████████| 18000/18000 [00:40<00:00, 440.62it/s]


In [51]:
for ch in range(4):
    ch_mean = np.mean(X_train[..., ch])
    ch_std = np.std(X_train[..., ch])
    print(ch, ch_mean, ch_std)
    X_train[...,ch] -= ch_mean
    X_train[...,ch] /= ch_std
    X_test[...,ch] -= ch_mean
    X_test[...,ch] /= ch_std


0 119.94996 40.376907
1 7735.771 4942.955
2 -0.0008432991 14.431183
3 0.0014509524 17.086712


In [63]:
for ch in range(1, 4):
    ch_max = np.max(X_train[..., ch])
    X_train[..., ch] /= ch_max
    X_test[..., ch] /= ch_max

In [44]:
X_train.shape, Y_train.shape, X_test.shape

((4000, 128, 128, 4), (4000, 128, 128, 1), (18000, 128, 128, 4))

In [64]:
np.save('input/X_train_filters.npy', X_train)
np.save('input/y_train_filters.npy', Y_train)
np.save('input/X_test_filters.npy', X_test)

In [3]:
def load_extra():
    depths_df = pd.read_csv('input/depths.csv')
    z_dict = depths_df.set_index('id').to_dict()['z']
    X_train = np.array([z_dict[x] for x in train_ids])
    X_test = np.array([z_dict[x] for x in test_ids])
    
    z_mean = np.mean(X_train)
    z_std = np.std(X_train)
    
    X_train -= z_mean
    X_train /= z_std
    
    X_test -= z_mean
    X_test /= z_std
    
    return X_train, X_test

X_train, X_test = load_extra()
